**handson用資料としての注意点**

普通、同じセル上で何度も試行錯誤するので、最終的に上手くいったセルしか残らず、失敗したセルは残りませんし、わざわざ残しません。

今回はhandson用に 試行・思考過程を残したいと思い、エラーやミスが出ても下のセルに進んで処理を実行するようにしています。

notebookのセル単位の実行ができるからこそのやり方かもしれません。良い。

（下のセルから文は常体で書きます。）

kunai (@jdgthjdg)

---


# 今までの復習 + 追加の分析ネタ

---

# AMeDASのデータを使って、天候との関連があるかを知りたい

* AMeDASのエクセルファイルも同様の前処理が必要  
* シートも複数ある

<br>

例
1. 降水量と花粉
1. 気温と花粉

---


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import japanize_matplotlib
import pandas as pd
import numpy as np
import qgrid
#設定でDataFrameなどが長く表示されないようにします（画面領域の消費を抑えてhandsonをしやすくするため）
# 長い場合の途中の省略表示(...)を出す閾値の設定（折り返しとは無関係）
pd.set_option('max_rows',10)
pd.set_option('max_columns',20) # これを超えたら全部は表示しない。 A B C ... X Y Z のように途中を省く。

In [2]:
kafun = pd.read_pickle("kafun04.pkl")
kafun.head(2)

,彦根市役所,滋賀県琵琶湖・環境科学研究センター,京都府立医科大学,京都府中丹東保健所,京都市右京区役所京北合同庁舎,豊中市役所第一庁舎,兵庫県立健康環境科学研究センター,兵庫県篠山庁舎,奈良県吉野保健所,和歌山地方気象台,和歌山県西牟婁振興局庁舎
date_hour,,,,,,,,,,,
2008-02-01 00:00:00,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2008-02-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,8.0,0.0


↓ ここからは01〜05のipynbファイルを見直しながら自由に記述！
---
